# Core

> The building blocks to the UI

## Imports

In [ ]:
#| default_exp core

In [ ]:
#| hide
#| export
import fasthtml.common as fh
from fasthtml.common import is_listy, Div, P, Span, Script, FastHTML, FT, to_xml, show
from fasthtml.svg import Svg
from enum import Enum
from fasthtml.components import Uk_select,Uk_input_tag
from functools import partial
from itertools import zip_longest
from typing import Union, Tuple, Optional
from fastcore.all import *
import copy
# from aenum import Enum

In [ ]:
%%html
<script src="https://cdn.tailwindcss.com"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"></script>
<script type="module" src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js"></script>
<link rel="stylesheet" href="https://unpkg.com/franken-wc@0.0.6/dist/css/blue.min.css">


<style>
#notebook-container { max-width: none; }
.output_html * { list-style-type: none !important; }
</style>

In [ ]:
#| hide
from functools import partial
from IPython.display import HTML

## Utils

### Docs

In [ ]:
#| hide
import pandas as pd

In [ ]:
#| hide
def show_enum_options(enum):
    df = pd.DataFrame([
        {'name': k,
         'value': v.value,
         'description': '' if v.__doc__ in [None, 'An enumeration.', enum.__doc__] else v.__doc__}
        for k, v in enum.__members__.items()
    ])
    if df['description'].str.strip().eq('').all():
        df = df.drop(columns=['description'])
    return df.style.hide(axis='index')

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| hide
show_doc = partial(show_doc, title_level=4)

### Tests

In [ ]:
#| hide
from fastcore.test import *

In [ ]:
#| hide
def test_xml_eq(a, b):
    "Tests if if 2 FT components generate the same xml"
    test_eq(*map(lambda x: x if isinstance(x,str) else to_xml(x), (a, b)))

### Stringification

In [ ]:
#| export
# need a better name, stringify might be too general for what it does 
def stringify(o # String, Tuple, or Enum options we want stringified
             ): # String that can be passed FT comp args (such as `cls=`)
    "Converts input types into strings that can be passed to FT components"  
    if is_listy(o): return ' '.join(map(str,o)) if o else ""
    return o.__str__()

In [ ]:
assert stringify('abc') == 'abc'
assert stringify(('abc','def')) == 'abc def'
assert 'uk-input ' + stringify(()) == 'uk-input '
assert 'uk-input ' + stringify("") == 'uk-input '

### Enum Helper

In [ ]:
#| export
#| hide
def str2ukcls(base, txt): 
    return f"uk-{base}-{txt.replace('_', '-')}".strip('-')

In [ ]:
#| export
#| hide
class VEnum(Enum): 
    def __add__(self, other):   return stringify((self, other))
    def __radd__(self, other):  return stringify((other, self)) 
    def __str__(self): return self.value
    
    def __new__(cls, value, doc=None):
        member = str.__new__(cls) 
        member._value_ = value
        if doc is not None:
            member.__doc__ = doc
        return member

In [ ]:
#| export
#| hide
def create_uk_enum(name, options, custom={}, enum_doc=None):
    def getval(a): return custom[a] if a in custom.keys() else a
    opts = {}
    for option in options:
        if isinstance(option, tuple): key, doc = option
        else: key, doc = option, None
        opts[key] = (str2ukcls(name.rstrip('T').lower(), getval(key)), doc)
    
    enum_class = VEnum(name, opts, type=str)
    if enum_doc:
        enum_class.__doc__ = enum_doc
    return enum_class


In [ ]:
#| hide
TestT = create_uk_enum('TestT',('foo','bar'))
stringify((TestT.foo,TestT.bar))

'uk-test-foo uk-test-bar'

In [ ]:
#| hide
# Test the fix
TestT = create_uk_enum('TestT', (('foo', 'foo docs'), ('bar', 'bar docs')))
print(stringify((TestT.foo, TestT.bar)))
print(TestT.foo.__doc__, TestT.bar.__doc__)

uk-test-foo uk-test-bar
foo docs bar docs


## Basic Components

In [ ]:
#| export
def UkGenericComponent(component_fn, *c, cls=(), **kwargs):
    " Create a new component based on a basic HTML component for use with FrankenUI"
    return component_fn(cls=cls, **kwargs)(*c)

### Button

In [ ]:
#| export
ButtonT = create_uk_enum('ButtonT',('default', 
                                    ('primary', 'primary color from theme'),
                                    ('secondary', 'Uses secondary color from theme'),
                                    ('danger', 'Red danger button'),
                                    'text',
                                    'link',
                                    ('ghost', 'Transparent style')),
                        enum_doc="Style Options for Button from https://franken-ui.dev/docs/button")

In [ ]:
show_doc(ButtonT)

---

#### ButtonT

>      ButtonT (*values)

*Style Options for Button from https://franken-ui.dev/docs/button*

In [ ]:
show_enum_options(ButtonT)

name,value,description
default,uk-button-default,
primary,uk-button-primary,primary color from theme
secondary,uk-button-secondary,Uses secondary color from theme
danger,uk-button-danger,Red danger button
text,uk-button-text,
link,uk-button-link,
ghost,uk-button-ghost,Transparent style


In [ ]:
#| export
def Button(
           *c:str|FT,                     # Components to go inside the Button
           cls:str|Enum=ButtonT.default,  # cls for the Button (see ButtonT for style options)
           **kwargs                       # any other kwargs will be passed to the button 
          )-> FT:                         # Button w/ `type=button` and `uk-button` cls
    "A Button with Uk Styling"
    return UkGenericComponent(fh.Button,*c, cls=('uk-button',stringify(cls)), type='button', **kwargs)

In [ ]:
show_doc(Button)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/core.py#L165){target="_blank" style="float:right; font-size:smaller"}

#### Button

>      Button (*c:str|fastcore.xml.FT, cls:str|enum.Enum=<ButtonT.default: 'uk-
>              button-default'>, **kwargs)

*A Button with Uk Styling*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| c | str \| fastcore.xml.FT |  |  |
| cls | str \| enum.Enum | uk-button-default |  |
| kwargs |  |  |  |
| **Returns** | **FT** |  | **Button w/ `type=button` and `uk-button` cls** |

In [ ]:
Button("Click Me")

```html
<button type="button" class="uk-button uk-button-default">Click Me</button>
```

In [ ]:
show(Div(Button("Default Class"),
        Button("Primary Class", cls=ButtonT.primary),
        Button("Secondary Class", cls=ButtonT.secondary),
        Button("Danger Class", cls=ButtonT.danger),
        Button("Ghost Class", cls=ButtonT.ghost),
        Button("Link Class", cls=ButtonT.link),
        Button("Text Class", cls=ButtonT.text),))

### Headings

In [ ]:
#| export
def H1(*c:FT|str,       # Components to go inside the Heading
       cls:Enum|str|tuple=(),   # cls for the Heading
       **kwargs  # any other kwargs will be passed to the Heading
      )->FT: # Heading with `class=uk-h1` cls
    "A H1 with Uk Styling"
    return UkGenericComponent(fh.H1, *c, cls=('uk-h1',stringify(cls)), **kwargs)

In [ ]:
show_doc(H1)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/core.py#L98){target="_blank" style="float:right; font-size:smaller"}

#### H1

>      H1 (*c:fastcore.xml.FT|str, cls:enum.Enum|str|tuple=(), **kwargs)

*A H1 with Uk Styling*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| c | fastcore.xml.FT \| str |  |  |
| cls | enum.Enum \| str \| tuple | () |  |
| kwargs |  |  |  |
| **Returns** | **FT** |  | **Heading with `class=uk-h1` cls** |

In [ ]:
#| export
def H2(*c:FT|str,       # Components to go inside the Heading
       cls:Enum|str|tuple=(),   # cls for the Heading
       **kwargs  # any other kwargs will be passed to the Heading
      )->FT: # Heading with `class=uk-h2` cls
    "A H2 with Uk Styling"
    return UkGenericComponent(fh.H2, *c, cls=('uk-h2',stringify(cls)), **kwargs)

In [ ]:
show_doc(H2)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/core.py#L106){target="_blank" style="float:right; font-size:smaller"}

#### H2

>      H2 (*c:fastcore.xml.FT|str, cls:enum.Enum|str|tuple=(), **kwargs)

*A H2 with Uk Styling*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| c | fastcore.xml.FT \| str |  |  |
| cls | enum.Enum \| str \| tuple | () |  |
| kwargs |  |  |  |
| **Returns** | **FT** |  | **Heading with `class=uk-h2` cls** |

In [ ]:
#| export
def H3(*c:FT|str,       # Components to go inside the Heading
       cls:Enum|str|tuple=(),   # cls for the Heading
       **kwargs  # any other kwargs will be passed to the Heading
      )->FT: # Heading with `class=uk-h3` cls
    "A H1 with Uk Styling"
    return UkGenericComponent(fh.H1, *c, cls=('uk-h1',stringify(cls)), **kwargs)

In [ ]:
show_doc(H3)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/core.py#L114){target="_blank" style="float:right; font-size:smaller"}

#### H3

>      H3 (*c:fastcore.xml.FT|str, cls:enum.Enum|str|tuple=(), **kwargs)

*A H1 with Uk Styling*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| c | fastcore.xml.FT \| str |  |  |
| cls | enum.Enum \| str \| tuple | () |  |
| kwargs |  |  |  |
| **Returns** | **FT** |  | **Heading with `class=uk-h3` cls** |

In [ ]:
#| export
def H4(*c:FT|str,       # Components to go inside the Heading
       cls:Enum|str|tuple=(),   # cls for the Heading
       **kwargs  # any other kwargs will be passed to the Heading
      )->FT: # Heading with `class=uk-h4` cls
    "A H4 with Uk Styling"
    return UkGenericComponent(fh.H4, *c, cls=('uk-H4',stringify(cls)), **kwargs)

In [ ]:
show_doc(H4)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/core.py#L122){target="_blank" style="float:right; font-size:smaller"}

#### H4

>      H4 (*c:fastcore.xml.FT|str, cls:enum.Enum|str|tuple=(), **kwargs)

*A H4 with Uk Styling*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| c | fastcore.xml.FT \| str |  |  |
| cls | enum.Enum \| str \| tuple | () |  |
| kwargs |  |  |  |
| **Returns** | **FT** |  | **Heading with `class=uk-h4` cls** |

In [ ]:
show(Div(
    H1("Heading 1 (H1)"),
    H2("Heading 2 (H2)"),
    H3("Heading 3 (H3)"),
    H4("Heading 4 (H4)")))

### Articles & Containers

In [ ]:
#| export
def Alert(*c, cls=(), **kwargs): 
    return UkGenericComponent(Div, *c, cls=('uk-alert',stringify(cls)), uk_alert=True, **kwargs)

In [ ]:
Alert()

```html
<div class="uk-alert " uk-alert></div>

```

In [ ]:
#| export
def Article(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Article, *c, cls=('uk-article',stringify(cls)), **kwargs)

def ArticleTitle(*c, cls=(), **kwargs):
    return UkGenericComponent(H1, *c, cls=('uk-article-title',stringify(cls)), **kwargs)

def ArticleMeta(*c, cls=(), **kwargs):
    return UkGenericComponent(P, *c, cls=('uk-article-meta',stringify(cls)), **kwargs)

In [ ]:
Article(ArticleTitle("Article Title"), ArticleMeta("By: John Doe"))

```html
<article class="uk-article ">
  <h1 class="uk-h1 uk-article-title ">Article Title</h1>
  <p class="uk-article-meta ">By: John Doe</p>
</article>

```

In [ ]:
show(Article(ArticleTitle("Article Title"), ArticleMeta("By: John Doe")))

In [ ]:
#| export
ContainerT = create_uk_enum('ContainerT',('xsmall','small','large','xlarge','expand'), enum_doc='Max width container sizes from https://franken-ui.dev/docs/container')

In [ ]:
show_doc(ContainerT)

---

#### ContainerT

>      ContainerT (*values)

*Max width container sizes from https://franken-ui.dev/docs/container*

In [ ]:
show_enum_options(ContainerT)

name,value
xsmall,uk-container-xsmall
small,uk-container-small
large,uk-container-large
xlarge,uk-container-xlarge
expand,uk-container-expand


In [ ]:
#| export
def Container(*c, cls=(), **kwargs): 
    return UkGenericComponent(Div, *c, cls=('uk-container',stringify(cls)), **kwargs)

In [ ]:
# Add background colors to the containers
show(Div(Container(cls=(ContainerT.xsmall, "bg-blue-200"))("XSmall container"),
         Container(cls=(ContainerT.small, "bg-red-200"))("Small container"),
         Container(cls=(ContainerT.large, "bg-yellow-200"))("Large container")))

### Forms & Inputs

In [ ]:
#| export
def Input(*c, cls=(), **kwargs): 
    return UkGenericComponent(fh.Input, *c, cls=('uk-input',stringify(cls)), **kwargs)
def Select(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Select, *c, cls=('uk-select',stringify(cls)), **kwargs)
def Radio(*c, cls=(), **kwargs): 
    return UkGenericComponent(fh.Input, *c, cls=('uk-radio',stringify(cls)), type='radio', **kwargs)
def CheckboxX(*c, cls=(), **kwargs): 
    return UkGenericComponent(fh.Input, *c, cls=('uk-checkbox',stringify(cls)), type='checkbox', **kwargs)
def Range(*c, cls=(), **kwargs): 
    return UkGenericComponent(fh.Input, *c, cls=('uk-range',stringify(cls)), type='range', **kwargs)
def Toggle_switch(*c, cls=(), **kwargs): 
    return UkGenericComponent(fh.Input, *c, cls=('uk-toggle-switch',stringify(cls)), type='checkbox', **kwargs)
def TextArea(*c, cls=(), **kwargs): 
    return UkGenericComponent(fh.Textarea, *c, cls=('uk-textarea',stringify(cls)), **kwargs)
def Button(*c, cls=ButtonT.default,  **kwargs):
    return UkGenericComponent(fh.Button,*c, cls=('uk-button',stringify(cls)), type='button', **kwargs)
def Switch(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Input, *c, cls=('uk-toggle-switch',stringify(cls)), type='checkbox', **kwargs)

In [ ]:
show(Div(
    P("Input: ", Input()),
    P("Select: ", Select(fh.Option("Option 1"), fh.Option("Option 2"))),
    P("Radio: ", Radio()),
    P("Checkbox: ", CheckboxX()),
    P("Range: ", Range()),
    P("Toggle switch: ", Toggle_switch()),
    P("TextArea: ", TextArea()),
    P("Button: ", Button("Click me")),
    P("Switch: ", Switch()),
    cls="space-y-4"
))

In [ ]:
#| export
LabelT = create_uk_enum('LabelT', ('primary', 'secondary', 'danger'))

In [ ]:
#| export
def Label(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Label, *c, cls=('uk-label',stringify(cls)), **kwargs)

In [ ]:
Label("My Label")

```html
<label class="uk-label ">My Label</label>
```

In [ ]:
show(Label("My Label"))

In [ ]:
#|export
def FormLabel(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Label, *c, cls=('uk-form-label',stringify(cls)), **kwargs)

In [ ]:
FormLabel('My Form Label')

```html
<label class="uk-form-label ">My Form Label</label>
```

In [ ]:
show(FormLabel('My Form Label'))

### Links

In [ ]:
#| export
LinkT = create_uk_enum('LinkT', ('muted', 'text', 'reset'), enum_doc='Link styles from https://franken-ui.dev/docs/link')

In [ ]:
show_doc(LinkT)

---

#### LinkT

>      LinkT (*values)

*Link styles from https://franken-ui.dev/docs/link*

In [ ]:
show_enum_options(LinkT)

name,value
muted,uk-link-muted
text,uk-link-text
reset,uk-link-reset


In [ ]:
#| export
def Link(*c, cls=(), **kwargs):  
    return UkGenericComponent(fh.A, *c, cls=('uk-link',stringify(cls)), **kwargs)

In [ ]:
Link('Default Link')

```html
<a href="#" class="uk-link ">Default Link</a>
```

In [ ]:
show(Div(cls='space-x-4')(Link('Default Link'),
                     Link('Muted Link', cls=LinkT.muted),
                     Link('Text Link', cls=LinkT.text),
                     Link('Reset Link', cls=LinkT.reset)))

### Lists

In [ ]:
ListT = create_uk_enum('ListT', (
    *map(lambda x: (x, "Style Modifier"),('disc', 'circle', 'square', 'decimal', 'hyphen')), 
    *map(lambda x: (x, "Color Modifier"),('muted', 'primary', 'secondary')),
    *map(lambda x: (x, "Other Modifier"),('bullet', 'divider', 'striped')),
), enum_doc='List styles from https://franken-ui.dev/docs/list')

In [ ]:
show_doc(ListT)

---

#### ListT

>      ListT (*values)

*List styles from https://franken-ui.dev/docs/list*

In [ ]:
show_enum_options(ListT)

name,value,description
disc,uk-list-disc,Style Modifier
circle,uk-list-circle,Style Modifier
square,uk-list-square,Style Modifier
decimal,uk-list-decimal,Style Modifier
hyphen,uk-list-hyphen,Style Modifier
muted,uk-list-muted,Color Modifier
primary,uk-list-primary,Color Modifier
secondary,uk-list-secondary,Color Modifier
bullet,uk-list-bullet,Other Modifier
divider,uk-list-divider,Other Modifier


In [ ]:
#| export
def List(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Ul, *c, cls=('uk-list',stringify(cls)), **kwargs)

In [ ]:
list_options = [(style,str(cls)) for style,cls in ListT.__members__.items()]

def create_list(style, cls):
    return List(fh.Li("Item 1"), fh.Li("Item 2"), cls=cls)

lists = [Div(H4(f"{style} List:"), create_list(style, cls)) for style, cls in list_options]
show(Div(*lists, cls="grid grid-cols-6"))

### Modal

In [ ]:
#| export
def ModalContainer(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Div, *c, cls=('uk-modal-container',stringify(cls)), uk_modal=True, **kwargs)

def ModalDialog(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Div, *c, cls=('uk-modal-dialog',stringify(cls)), **kwargs)

def ModalHeader(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Div, *c, cls=('uk-modal-header',stringify(cls)), **kwargs)

def ModalBody(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Div, *c, cls=('uk-modal-body',stringify(cls)), **kwargs)

def ModalFooter(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Div, *c, cls=('uk-modal-footer',stringify(cls)), **kwargs)

def ModalTitle(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.H2, *c, cls=('uk-modal-title',stringify(cls)), **kwargs)

def ModalCloseButton(*c, cls=(), **kwargs):
    return UkGenericComponent(Button, *c, cls=('uk-modal-close',stringify(cls)), **kwargs)

In [ ]:
ModalCloseButton('test', cls=ButtonT.ghost)

```html
<button type="button" class="uk-button uk-modal-close uk-button-ghost">test</button>
```

In [ ]:
#| export
def Modal(*c,
        header=None,          # Components that go in the header
        footer=None,          # Components that go in the footer
        cls=(),               # class for outermost container
        dialog_cls=(),        # classes for the dialog
        header_cls='p-6',     # classes for the header
        body_cls='space-y-6', # classes for the body
        footer_cls=(),        # classes for the footer
        id='',                # id for the outermost container
        **kwargs              # classes for the outermost container
        ):
    cls, dialog_cls, header_cls, body_cls, footer_cls = map(stringify, (cls, dialog_cls, header_cls, body_cls, footer_cls))
    res = []
    if header: res.append(ModalHeader(cls=header_cls)(header))
    res.append(ModalBody(cls=body_cls)(*c))
    if footer: res.append(ModalFooter(cls=footer_cls)(footer))
    return ModalContainer(ModalDialog(*res, cls=dialog_cls), cls=cls, id=id, **kwargs)

### Navs

In [ ]:
#| export
def Nav(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Ul, *c, cls=('uk-nav',stringify(cls)), **kwargs)

In [ ]:
#| export
def NavBarContainer(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Div, *c, cls=('uk-navbar-container',stringify(cls)), uk_navbar=True, **kwargs)

def NavBarNav(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Nav, *c, cls=('uk-navbar-nav',stringify(cls)), **kwargs)

In [ ]:
#| export
PaddingT = create_uk_enum('PaddingT', (
    *map(lambda x: (x, "Size Modifier"),('xsmall', 'small', 'default', 'medium', 'large', 'xlarge')),
    *map(lambda x: (x, "Remove Side Mofiers"),('remove', 'remove_top', 'remove_bottom', 'remove_left', 'remove_right', 'remove_vertical', 'remove_horizontal'))
), enum_doc="Padding Modifiers from https://franken-ui.dev/docs/padding", custom={'default':''})

In [ ]:
show_doc(PaddingT)

---

#### PaddingT

>      PaddingT (*values)

*Padding Modifiers from https://franken-ui.dev/docs/padding*

In [ ]:
show_enum_options(PaddingT)

name,value,description
xsmall,uk-padding-xsmall,Size Modifier
small,uk-padding-small,Size Modifier
default,uk-padding,Size Modifier
medium,uk-padding-medium,Size Modifier
large,uk-padding-large,Size Modifier
xlarge,uk-padding-xlarge,Size Modifier
remove,uk-padding-remove,Remove Side Mofiers
remove_top,uk-padding-remove-top,Remove Side Mofiers
remove_bottom,uk-padding-remove-bottom,Remove Side Mofiers
remove_left,uk-padding-remove-left,Remove Side Mofiers


In [ ]:
#| export
PositionT = create_uk_enum('PositionT', (
    'top', 'bottom', 'left', 'right','top_left', 'top_center', 'top_right','center', 'center_left', 'center_right',
    'bottom_left', 'bottom_center', 'bottom_right','center_horizontal', 'center_vertical'))

In [ ]:
#| export
def Placeholder(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Div, *c, cls=('uk-placeholder',stringify(cls)), **kwargs)

In [ ]:
#| export
def Progress(*c, cls=(), value="", max="", **kwargs):
    return UkGenericComponent(fh.Progress, *c, value=value, max=max, cls=('uk-progress',stringify(cls)), **kwargs)

In [ ]:
#| export
SectionT = create_uk_enum('SectionT', (
    'default', 'muted', 'primary', 'secondary',
    'xsmall', 'small', 'large', 'xlarge', 'remove_vertical'
))

In [ ]:
#| export
def Section(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Div, *c, cls=('uk-section',stringify(cls)), **kwargs)

In [ ]:
#| export
def Sticky(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Div, *c, cls=stringify(cls), uk_sticky=True, **kwargs)

## Theme

You can select a theme color to and get all the headers.  This brings in the most standard grouping:
+ Tailwind: Optional, but typically FrankenUI is used in conjunction with tailwind
+ Uikit: Many things are styled with uikit
+ UIkit icons: Gives many icons OOTB, and is nice to have handy.
+ Franken WC: Neccesary for some of the (minimal) interactivity components have

In [ ]:
#| export
class Theme(Enum):
    slate = "slate"
    stone = "stone"
    gray = "gray"
    neutral = "neutral"
    red = "red"
    rose = "rose"
    orange = "orange"
    green = "green"
    blue = "blue"
    yellow = "yellow"
    violet = "violet"
    zinc = "zinc"

    def headers(self):
        js = (Script(src="https://cdn.tailwindcss.com"),
              Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"),
              Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"),
              Script(type="module", src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js")
              )
        _url = f"https://unpkg.com/franken-wc@0.0.6/dist/css/{self.value}.min.css"
        return (*js, Link(rel="stylesheet", href=_url))

In [ ]:
hdrs = Theme.blue.headers()
app = FastHTML(hdrs=hdrs)

## Text Style

In [ ]:
#| export
TextT = create_uk_enum('TextT', (
    'lead', 'meta', 'italic', # Style
    'small', 'default', 'large', #Font Size
    'light', 'normal', 'bold', 'lighter', 'bolder', # Font Weight
    'capitalize', 'uppercase', 'lowercase', #Text transform
    'decoration_none', # Text decoration
    'muted', 'primary', 'secondary', 'success', 'warning', 'danger', # Color
    'left', 'right', 'center', 'justify', # Alignment
    'top', 'middle', 'bottom', 'baseline', # Vertical alignment
    'truncate', 'break_', 'nowrap' # Wrapping
))

In [ ]:
#| export
class TextFont(Enum):
    def __add__(self, other):   return stringify((self, other))
    def __radd__(self, other):  return stringify((other, self)) 
    def __str__(self): return self.value
    muted_sm = stringify((TextT.muted, TextT.small))
    bold_sm = stringify((TextT.bold, TextT.small))

In [ ]:
show_enum_options(TextFont)

name,value
muted_sm,uk-text-muted uk-text-small
bold_sm,uk-text-bold uk-text-small


## Icons and Avatars

In [ ]:
#| export
def UkIcon(icon,    # Icon name from https://getuikit.com/docs/icon
           ratio=1, # Icon ratio/size 
           cls=()   # Span classes
          ):        # Span with Icon
    "Creates a Span with the given icon"
    return Span(uk_icon=f"icon: {icon}; ratio: {ratio}",cls=stringify(cls))

In [ ]:
UkIcon('heart', 0.5)

```html
<span uk-icon="icon: heart; ratio: 0.5"></span>
```

In [ ]:
#| export
# def Img(*args, data_src="", cls=(), **kwargs):
#     return UkGenericComponent(fh.Div, *args, data_src=data_src, uk_img=True, cls=stringify(cls), **kwargs)

In [ ]:
#| export
def DiceBearAvatar(seed_name, # Seed name (ie 'Isaac Flath')
                   h,         # Height 
                   w          # Width
                  ):          # Span with Avatar
    url = 'https://api.dicebear.com/8.x/lorelei/svg?seed='
    return Span(cls=f"relative flex h-{h} w-{w} shrink-0 overflow-hidden rounded-full bg-accent")(
            fh.Img(cls="aspect-square h-full w-full", alt="Avatar", src=f"{url}{seed_name}"))

## Spacing | Flexbox | Grid

The most common pattern for spacing it to organize the general high level page layout with a `Grid`, and smaller components with `Flex`.

:::{.callout-tip}
Play [Flex Box Froggy](https://flexboxfroggy.com/) to get an understanding of flex box.
:::

In [ ]:
#| export
FlexT = create_uk_enum('FlexT', (
    'block', 'inline',
    'left', 'center', 'right', 'between', 'around', # horizontal
    'stretch', 'top', 'middle', 'botton', # Vertical
    'row', 'row_reverse', 'col', 'col_reverse', # Direction
    'nowrap', 'wrap', 'wrap_reverse' # Wrap
), {'block':''})

In [ ]:
Div(cls=(FlexT.block,FlexT.left,FlexT.middle,'some_cls'))

```html
<div class="uk-flex uk-flex-left uk-flex-middle some_cls"></div>

```

In [ ]:
#| export
GridT = create_uk_enum('GridT', ('small', 'medium', 'large', 'collapse'))

In [ ]:
#|export
def Grid(*c,      # Divs/Containers that should be divided into a grid
         cols=3,  # Number of columns
         cls=GridT.small,  # Additional classes for Grid Div
         **kwargs # Additional args for Grid Div
        ):
    """Creates a grid with the given number of columns, often used for a grid of cards"""
    cls = stringify(cls)
    return Div(cls=(f'grid grid-cols-{cols}',cls), **kwargs)(*c)

In [ ]:
Grid(*map(Div, range(6)),cols=3, cls=GridT.small)

```html
<div class="grid grid-cols-3 uk-grid-small">
  <div>0</div>
  <div>1</div>
  <div>2</div>
  <div>3</div>
  <div>4</div>
  <div>5</div>
</div>

```

In [ ]:
#| export
def ResponsiveGrid(*c, sm=1, md=2, lg=3, xl=4, gap=2, cls='', **kwargs):
    "Creates a responsive grid with the given number of columns for different screen sizes"
    return Div(cls=f'grid grid-cols-{sm} md:grid-cols-{md} lg:grid-cols-{lg} xl:grid-cols-{xl} gap-{gap} ' + stringify(cls), **kwargs)(*c)

We can use the `FlexT` enum above to create `Div`s that arrange components in a flex box in different ways.  

For example, `FullySpacedDiv` applies a very common pattern so we've made a function that helps to this for you.  But if you look at the code, it's really just using the enum to apply the right combonation of flex classes.

In [ ]:
#| exports
def FullySpacedDiv(*c,                # Components
                   cls='uk-width-1-1',# Classes for outer div
                   **kwargs           # Additional args for outer div
                  ):                  # Div with spaced components via flex classes
    "Creates a flex div with it's components having as much space between them as possible"
    cls = stringify(cls)
    return Div(cls=(FlexT.block,FlexT.between,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
FullySpacedDiv(*map(P, range(3)))

```html
<div class="uk-flex uk-flex-between uk-flex-middle uk-width-1-1">
  <p>0</p>
  <p>1</p>
  <p>2</p>
</div>

```

In [ ]:
#| export
def CenteredDiv(*c,      # Components
                cls=(),  # Classes for outer div
                **kwargs # Additional args for outer div
               ): # Div with components centered in it
    "Creates a flex div with it's components centered in it"
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.col,FlexT.middle,FlexT.center,cls),**kwargs)(*c)

In [ ]:
CenteredDiv(*map(P, range(3)))

```html
<div class="uk-flex uk-flex-col uk-flex-middle uk-flex-center ">
  <p>0</p>
  <p>1</p>
  <p>2</p>
</div>

```

In [ ]:
#| export
def LAlignedDiv(*c,      # Components
                cls=(),  # Classes for outer div
                **kwargs # Additional args for outer div
               ): # Div with components aligned to the left
    "Creates a flex div with it's components aligned to the left"
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.left,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
LAlignedDiv(*map(P, range(3)))

```html
<div class="uk-flex uk-flex-left uk-flex-middle ">
  <p>0</p>
  <p>1</p>
  <p>2</p>
</div>

```

In [ ]:
#| export
def RAlignedDiv(*c,      # Components
                cls=(),  # Classes for outer div
                **kwargs # Additional args for outer div
               ): # Div with components aligned to the right
    "Creates a flex div with it's components aligned to the right"
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.right,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
RAlignedDiv(*map(P, range(3)))

```html
<div class="uk-flex uk-flex-right uk-flex-middle ">
  <p>0</p>
  <p>1</p>
  <p>2</p>
</div>

```

In [ ]:
#| export
def VStackedDiv(*c, cls='', **kwargs):
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.col,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
#| export
def HStackedDiv(*c, cls='', **kwargs):
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.row,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
#| export
def SpaceBetweenDiv(*c, cls='', **kwargs):
    cls = stringify(cls)
    return Div(cls=(FlexT.block,FlexT.between,FlexT.middle,cls), **kwargs)(*c)

## Basic Inputs

Inputs of various types often go with a label.  Because of this we created functions to do this for you along with properly linking the `for` attribute from the lable to the input.  We also have some nice defaults, such as putting a little spacing between the label and the input

In [ ]:
#| export
def GenericLabelInput(
               label:str|FT,
               input_fn, 
               lbl_cls='',
               input_cls='',
               container=Div, 
               container_cls='',
               id='',
                **kwargs
                ):
    "`Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `UkTextArea` which is created for you in this library"
    if isinstance(label, str) or label.tag != 'label': 
        label = FormLabel(cls=stringify(lbl_cls), fr=id)(label)
    inp = input_fn(id=id, cls=stringify(input_cls), **kwargs)        
    if container: return container(label, inp, cls=stringify(container_cls))
    return label, inp

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn'])
def LabelInput(*args, **kwargs): return GenericLabelInput(*args, input_fn=Input, **kwargs)

In [ ]:
fh.show(LabelInput('mylabel',container=Form))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn'])
def LabelRadio(*args, **kwargs): return GenericLabelInput(*args, input_fn=Radio, **kwargs)

In [ ]:
fh.show(LabelRadio('mylabel',container=Form))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn'])
def LabelCheckboxX(*args, **kwargs): return GenericLabelInput(*args, input_fn=CheckboxX, **kwargs)

In [ ]:
fh.show(LabelCheckboxX('mylabel',container=Form))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn'])
def LabelRange(*args, **kwargs): return GenericLabelInput(*args, input_fn=Range, **kwargs)

In [ ]:
fh.show(LabelRange('mylabel',container=Form))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn'])
def LabelToggle_switch(*args, **kwargs): return GenericLabelInput(*args, input_fn=Toggle_switch, **kwargs)

In [ ]:
fh.show(Toggle_switch('mylabel',container=Form))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn'])
def LabelTextArea(*args, **kwargs): return GenericLabelInput(*args, input_fn=TextArea, **kwargs)

In [ ]:
fh.show(LabelTextArea('mylabel',container=Form))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn'])
def LabelSwitch(*args, **kwargs): return GenericLabelInput(*args, input_fn=Switch, **kwargs)

In [ ]:
fh.show(LabelSwitch('mylabel',container=Form))

In [ ]:
#| export
def LabelSelect(*option,
               label:str|FT,
               lbl_cls='',
               input_cls='',
               container=Div, 
               container_cls='',
               id='',
                **kwargs
                ):
    "`Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `UkTextArea` which is created for you in this library"
    if isinstance(label, str) or label.tag != 'label': 
        label = FormLabel(lbl_cls=stringify(lbl_cls), fr=id)(label)
    inp = Select(*option, id=id, cls=stringify(input_cls), **kwargs)        
    if container: return container(label, inp, cls=stringify(container_cls))
    return label, inp

In [ ]:
fh.show(LabelSelect(Option(1),Option(2),label='mylabel',container=Form))

## Buttons

In [ ]:
Button('MyButton', cls=ButtonT.primary)

```html
<button type="button" class="uk-button uk-button-primary">MyButton</button>
```

In [ ]:
#| export
def UkIconButton(*c, sz='small', cls=(), **kwargs):
    "Creates an `IconButton` with uk styling"
    if sz not in ('small','medium','large'): raise ValueError(f"Invalid size '{sz}'. Must be 'small', 'medium', or 'large'.")
    return Button(cls=f'uk-icon-button uk-icon-button-{sz} ' + stringify(cls), **kwargs)(*c)

In [ ]:
UkIconButton(UkIcon('home'))

```html
<button type="button" class="uk-button uk-icon-button uk-icon-button-small "><span uk-icon="icon: home; ratio: 1"></span></button>
```

## Drop Downs

In [ ]:
#| export
def Options(*c,                    # Content for an `Option`
            selected_idx:int=None, # Index location of selected `Option`
            disabled_idxs:set=None # Idex locations of disabled `Options`
           ):
    "Helper function to wrap things into `Option`s for use in `UkSelect`"
    return [fh.Option(o,selected=i==selected_idx, disabled=disabled_idxs and i in disabled_idxs) for i,o in enumerate(c)]

In [ ]:
list(map(to_xml,Options('option1','option2','option3', selected_idx=1, disabled_idxs={0,2})))

['<option disabled>option1</option>',
 '<option selected>option2</option>',
 '<option disabled>option3</option>']

In [ ]:
#| export
def UkSelect(*option,            # Options for the select dropdown (can use `Options` helper function to create)
             label=(),           # String or FT component for the label
             lbl_cls=(),         # Additional classes for the label
             inp_cls=(),         # Additional classes for the select input
             cls=('space-y-2',), # Classes for the outer div
             id="",              # ID for the select input
             name="",            # Name attribute for the select input
             placeholder="",     # Placeholder text for the select input
             searchable=False,   # Whether the select should be searchable
             **kwargs):          # Additional arguments passed to Uk_select
    "Creates a select dropdown with uk styling"
    lbl_cls, inp_cls, cls = map(stringify, (lbl_cls, inp_cls, cls))
    if label: 
        lbl = FormLabel(cls=f'{lbl_cls}', fr=id)(label) 
    select = Uk_select(*option, cls=inp_cls, uk_cloak=True, id=id, 
                       name=name, placeholder=placeholder, searchable=searchable, **kwargs)
    return Div(cls=cls)(lbl, select) if label else Div(cls=cls)(select)

In [ ]:
fh.show(UkSelect(*Options('a', 'b'), label='MyLabel', placeholder='MyPlaceholder'))

In [ ]:
fh.show(Div(cls='space-y-2')(LabelSelect(*Options('a', 'b'), label='MyLabel', placeholder='MyPlaceholder')))

In [ ]:
#| export
def _UkDropdownButtonOptions(opt_grps, opt_hdrs=None):
    res = []
    for g,h in zip_longest(opt_grps, tuplify(opt_hdrs)):
        if h: res.append(Li(cls="uk-nav-header")(h if isinstance(h,FT) else Div(h)))
        if isinstance(g,(list,tuple)): res += list(map(Li, g))
        else: res.append(Li(g))
    return res

In [ ]:
#| export
def UkDropdownButton(
    *opt_grp,        # List of options to be displayed in the dropdown
    opt_hdrs=None,  # List of headers for each option group, or None
    label=None,     # String, FT component, or None for the `Button`
    btn_cls=ButtonT.default,  # Button class(es)
    cls=(),         # Parent div class
    dd_cls=(),      # Class that goes on the dropdown container
    icon='triangle-down',  # Icon to use for the dropdown
    icon_cls='',    # Additional classes for the icon
    icon_position='right'  # Position of the icon: 'left' or 'right'
    ):
    dd_cls, btn_cls, cls, icon_cls = map(stringify, (dd_cls, btn_cls, cls, icon_cls))
    icon_component = UkIcon(icon, cls=icon_cls) if icon else None
    btn_content = [] if label is None else [label]
    if icon_component: btn_content.insert(0 if icon_position == 'left' else len(btn_content), icon_component)
    btn = Button(cls=btn_cls)(*btn_content)
    dd = Div(uk_dropdown='mode: click; pos: bottom-right', cls='uk-dropdown uk-drop ' + dd_cls)(
        Ul(cls='uk-dropdown-nav')(*_UkDropdownButtonOptions(opt_grp, opt_hdrs)))
    return Div(cls=cls)(Div(cls='flex items-center space-x-4')(btn, dd))

In [ ]:
fh.show(UkDropdownButton((A('Option 1.1', _href='#'), A('Option 1.2', _href='#')), 
                 (A('Option 2.1', _href='#'), A('Option 2.2', _href='#')),
                 opt_hdrs=('Header 1', 'Header 2')))

In [ ]:
UkIcon('home')

```html
<span uk-icon="icon: home; ratio: 1"></span>
```

## Dividers

In [ ]:
#| export
def UkHSplit(*c, cls=(), line_cls=(), text_cls=()):
    # Divider FrankenUI stuff
    cls, line_cls, text_cls = map(stringify,(cls, line_cls, text_cls))
    return Div(cls='relative ' + cls)(
        Div(cls="absolute inset-0 flex items-center " + line_cls)(Span(cls="w-full border-t border-border")),
        Div(cls="relative flex justify-center " + text_cls)(Span(cls="bg-background px-2 ")(*c)))

In [ ]:
UkHSplit()

```html
<div class="relative ">
  <div class="absolute inset-0 flex items-center ">
<span class="w-full border-t border-border"></span>  </div>
  <div class="relative flex justify-center ">
<span class="bg-background px-2 "></span>  </div>
</div>

```

In [ ]:
#| export
def UkHLine(lwidth=2, y_space=4): return Div(cls=f"my-{y_space} h-[{lwidth}px] w-full bg-secondary")

In [ ]:
UkHLine()

```html
<div class="my-4 h-[2px] w-full bg-secondary"></div>

```

In [ ]:
#| export
def UkNavDivider(): return fh.Li(cls="uk-nav-divider")

In [ ]:
UkNavDivider()

```html
<li class="uk-nav-divider"></li>

```

## NavBar

In [ ]:
#| export
def UkNavbarDropdown(*c, label, href='#', cls='', has_header=False, **kwargs):
    fn = lambda x: Li(item, cls='uk-drop-close', href='#demo', uk_toggle=True)
    flattened = []
    for i, item in enumerate(c):
        if i > 0: flattened.append(Li(cls="uk-nav-divider"))
        if isinstance(item, (list,tuple)): flattened.extend(map(Li, item))
        else: flattened.append(Li(item, cls="uk-nav-header" if i == 0 and has_header else None, uk_toggle=True))
    return (fh.Li(cls=cls, **kwargs)(
                fh.A(label, cls='uk-drop-close', href='#', uk_toggle=True), 
                Div(cls='uk-navbar-dropdown', uk_dropdown="mode: click; pos: bottom-left")(fh.Ul(cls='uk-nav uk-dropdown-nav')(*flattened))))

In [ ]:
UkNavbarDropdown(label='abc')

```html
<li>
<a href="#" uk-toggle class="uk-drop-close">abc</a>  <div class="uk-navbar-dropdown" uk-dropdown="mode: click; pos: bottom-left">
    <ul class="uk-nav uk-dropdown-nav"></ul>
  </div>
</li>

```

In [ ]:
#| export
def _NavBarSide(n, s):
    def add_class(item):
        if isinstance(item, str): return Li(cls='uk-navbar-item')(item)
        else: item.attrs['class'] = f"{item.attrs.get('class', '')} uk-navbar-item".strip()
        return item
    return Div(cls=f'uk-navbar-{s}')(Ul(cls='uk-navbar-nav')(*map(add_class, tuplify(n))))

In [ ]:
#| export
def UkNavbar(lnav: fh.Sequence[Union[str, FT]]=None, 
             rnav: fh.Sequence[Union[str, FT]]=None, 
             cls='',
             **kwargs
            ) -> FT:
    return Div(cls='uk-navbar-container uk-width-1-1 relative z-10 '+ stringify(cls), uk_navbar=True, **kwargs)(
             _NavBarSide(lnav,'left') if lnav else '',
             _NavBarSide(rnav,'right') if rnav else '')

In [ ]:
UkNavbar()

```html
<div class="uk-navbar-container uk-width-1-1 relative z-10 " uk-navbar>
</div>

```

## Sidebar

In [ ]:
sidebar_items = ["Profile", "Account", "Appearance", "Notifications", "Display"]
tst = fh.Ul(cls='uk-nav-primary', uk_switcher="connect: #component-nav; animation: uk-animation-fade")(
                            *map(lambda x: fh.Li(fh.A(x)),sidebar_items))

In [ ]:
#| export
def UkSidebar(*ul,                 # Each Ul can be it's own section.  Use A for links!
              cls='space-y-4 p-4', # Classes for outer container
              **kwargs             # Kwargs for outer container
             ):
    "Creates a styled sidebar component"
    styles = ('uk-nav-default', 'uk-nav-primary','uk-nav-secondary')
    sidebar = []
    for section in tuplify(ul):
        section = copy.deepcopy(section)
        _sattrs = section.attrs
        if 'class' not in _sattrs: _sattrs['class'] = ''
        if 'uk-nav' not in _sattrs: _sattrs['class'] += ' uk-nav '
        if not any(x in styles for x in _sattrs['class'].split()): _sattrs['class'] += ' uk-nav-default '
        sidebar.append(section)  
    return Div(cls=cls, **kwargs)(*sidebar)

In [ ]:
sidebar_items = ["Profile", "Account"]
tst = fh.Ul(cls='uk-nav-primary')(*map(lambda x: fh.Li(fh.A(x)),sidebar_items))
UkSidebar(tst)

```html
<div class="space-y-4 p-4">
  <ul class="uk-nav-primary uk-nav ">
    <li>
<a href="#">Profile</a>    </li>
    <li>
<a href="#">Account</a>    </li>
  </ul>
</div>

```

## Navtab

In [ ]:
#| export
def NavTab(text, active=False):
    return Li(cls="uk-active" if active else " ")(A(text, href="#demo", uk_toggle=True))

def UkTab(*items, maxw=96, cls='', **kwargs):
    cls = stringify(cls)
    return Ul(cls=f"uk-tab-alt max-w-{maxw} "+cls,**kwargs)(*[NavTab(item, active=i==0) for i, item in enumerate(items)])

## Cards

In [ ]:
#| export
CardT = create_uk_enum('CardT',('default', 'primary', 'secondary', 'danger'))

In [ ]:
#| export
def CardTitle(*c, cls=(), **kwargs):
    return UKGenericComponent(fh.Div, *c, cls=('uk-card-title',stringify(cls)), **kwargs)

def CardHeader(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Div, *c, cls=('uk-card-header',stringify(cls)), **kwargs)

def CardBody(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Div, *c, cls=('uk-card-body',stringify(cls)), **kwargs)

def CardFooter(*c, cls=(), **kwargs):
    return UkGenericComponent(fh.Div, *c, cls=('uk-card-footer',stringify(cls)), **kwargs)

def CardContainer(*c, cls=CardT.default, **kwargs):
    return UkGenericComponent(fh.Div, *c, cls=('uk-card',stringify(cls)), **kwargs)

In [ ]:
#|export
def Card(*c, # Components that go in the body
        header=None, # Components that go in the header
        footer=None,  # Components that go in the footer
        body_cls='space-y-6', # classes for the body
        header_cls=(), # classes for the header
        footer_cls=(), # classes for the footer
        cls=(), #class for outermost component
        **kwargs # classes that for the card itself
        ):
    header_cls, footer_cls, body_cls, cls = map(stringify, (header_cls, footer_cls, body_cls, cls))
    res = []
    if header: res.append(CardHeader(cls=header_cls)(header))
    res.append(CardBody(cls=body_cls)(*c))
    if footer: res.append(CardFooter(cls=footer_cls)(footer))
    return CardContainer(cls=cls, **kwargs)(*res)

## Tables

In [ ]:
#| export
TableT = create_uk_enum("TableT", ('divider', 'striped', 'hover', 'small', 'large', 
                                    'justify', 'middle','responsive',))

In [ ]:
#| export
def Table(*args, cls=(TableT.middle, TableT.divider, TableT.hover, TableT.small), **kwargs): 
    return fh.Table(cls=('uk-table', stringify(cls)), *args, **kwargs)

In [ ]:
#| export
def _TableCell(Component, *args, cls=(), shrink=False, expand=False, small=False, **kwargs):
    cls = stringify(cls)
    if shrink: cls += ' uk-table-shrink'
    if expand: cls += ' uk-table-expand'
    if small: cls += ' uk-table-small'
    return Component(*args,cls=cls, **kwargs)

@delegates(_TableCell, but=['Component'])
def Td(*args,**kwargs):  return _TableCell(fh.Td, *args, **kwargs)
@delegates(_TableCell, but=['Component'])
def Th(*args,**kwargs): return _TableCell(fh.Th, *args, **kwargs)

def Tr(*cells, cls=(), **kwargs):  return fh.Tr(*cells, cls=stringify(cls), **kwargs)
def Thead(*rows, cls=(), **kwargs): return fh.Thead(*rows, cls=stringify(cls), **kwargs)
def Tbody(*rows, cls=(), **kwargs): return fh.Tbody(*rows, cls=stringify(cls), **kwargs)

In [ ]:
#|export
def TableFromLists(header_data, body_data, footer_data=None, 
                   header_cell_render=Th,body_cell_render=Td, footer_cell_render=Td,
                   cls=(TableT.middle, TableT.divider, TableT.hover, TableT.small), **kwargs):
    
    return Table(
                Thead(Tr(*map(header_cell_render, header_data))),
                Tbody(*[Tr(*map(body_cell_render, r)) for r in body_data]),
                Tfoot(Tr(*map(footer_cell_render, footer_data))) if footer_data else '',
                cls=stringify(cls),    
                **kwargs)

In [ ]:
# Test the function
header = ['Name', 'Age', 'City']
body = [
    ['Alice', '25', 'New York'],
    ['Bob', '30', 'San Francisco'],
    ['Charlie', '35', 'London']
]
footer = ['Total', '90', '']

fh.show(TableFromLists(header, body, footer))

Name,Age,City
Alice,25,New York
Bob,30,San Francisco
Charlie,35,London
Total,90,


In [ ]:
#| export
def TableFromDicts(header_data:Sequence, body_data:Sequence[dict], footer_data=None, 
                   header_cell_render=Th, body_cell_render=lambda k,v : Td(v), footer_cell_render=lambda k,v : Td(v),
                   cls=(TableT.middle, TableT.divider, TableT.hover, TableT.small), **kwargs):
    
    return Table(
        Thead(Tr(*[header_cell_render(h) for h in header_data])),
        Tbody(*[Tr(*[body_cell_render(k, r) for k in header_data]) for r in body_data]),
        Tfoot(Tr(*[footer_cell_render(k, footer_data.get(k.lower(), '')) for k in header_data])) if footer_data else '',
        cls=stringify(cls),    
        **kwargs
    )

In [ ]:
# Example usage:
def header_render(v): return Th(v.upper())
def body_render(k, v):
    match k.lower():
        case 'name': return Td(v['name'], cls='font-bold')
        case 'age': return Td(f"{v['age']} years")
        case _: return Td(v[k.lower()])

header_data = ['Name', 'Age', 'City']
body_data = [
    {'name': 'Alice', 'age': 30, 'city': 'New York'},
    {'name': 'Bob', 'age': 25, 'city': 'London'}
]

fh.show(TableFromDicts(header_data, body_data, header_cell_render=header_render, body_cell_render=body_render))

NAME,AGE,CITY
Alice,30 years,New York
Bob,25 years,London


## Forms

In [ ]:
#| export
def UkFormSection(title, description, *c, button_txt='Update', outer_margin=6, inner_margin=6):
    return Div(cls=f'space-y-{inner_margin} py-{outer_margin}')(
        Div(H3(title), P(description, cls=TextFont.bold_sm)),
        UkHSplit(), *c,
        Div(Button(button_txt, cls=ButtonT.primary)) if button_txt else None)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
%%html
<script>
window.addEventListener('load', function() {
    for (let i = 1; i <= 6; i++) {
        document.querySelectorAll(`h${i}`).forEach(el => {
            el.classList.add(`uk-h${i}`);
        });
    }
});
</script>